## Ablation of number of heads used in all-tokens last layer attention probes
This notebook generates results for the ablation study of the number of attention heads used in all-tokens last layer attention probes.

In [ ]:
import pandas as pd
from constants import BASE_PATH_PROJECT, FOLDER_SUBSTRING

In [ ]:
new_df = pd.read_pickle(BASE_PATH_PROJECT/f"results_{FOLDER_SUBSTRING}_aat_diff_nr_heads/aggregated/all_runs_rebuttal.pkl")
ref_df = pd.read_pickle(BASE_PATH_PROJECT/f"results_{FOLDER_SUBSTRING}_exp/aggregated/all_runs_v11.pkl")

In [3]:
ref_df['contains_intermediate'] = ref_df['model_ids'].apply(lambda x: len([ elem for elem in eval(x) if elem.split("@")[-1] not in ["norm", "visual"]])>0)
new_df['contains_intermediate'] = new_df['model_ids'].apply(lambda x: len([ elem for elem in eval(x) if elem.split("@")[-1] not in ["norm", "visual"]])>0)

In [4]:
df = pd.concat([ref_df, new_df]).reset_index(drop=True)
df.shape

(1430, 86)

In [5]:
df = df[df['experiment'] == 'All tokens last layer']
df = df[df['model_size'] == 'base']
df = df[~df['contains_intermediate']]
df = df[df['dataset'].isin(['wds/fer2013', 'wds/gtsrb', 'wds/vtab/cifar100',  'wds/vtab/eurosat'])].copy().reset_index(drop=True)
df.shape

(48, 86)

In [27]:
single_linear_probe_res = df[df.num_heads == 8].copy()
single_linear_probe_res = single_linear_probe_res.set_index(['dataset', 'base_model'])

In [28]:
df['relative_test_bal_acc'] = df.apply(
    lambda x: (x['test_lp_bal_acc1'] - single_linear_probe_res.loc[(x['dataset'], x['base_model']), 'test_lp_bal_acc1'])*100, 
    axis=1
)

In [29]:
round(df.groupby(['num_heads'])['relative_test_bal_acc'].agg(['mean', 'std']), 4)

,mean,std
num_heads,,
8.0,0.0000,0.0000
12.0,0.3304,0.2657
16.0,0.1127,0.5147
32.0,0.6121,0.6333
